In [2]:
#pip install datasets

# An Endpoint using BERT large model (uncased) with the MS_MARCO dataset

---

## Background
Bidirectional Encoder Representations from Transformers (BERT) is a pre-trained language model that 
has achieved state-of-the-art results on numerous natural language processing (NLP) tasks. BERT is a 
transformer-based model, which was fine-tuned for various NLP tasks, including question answering. 
The Stanford Question Answering Dataset (SQuAD) is one such task where BERT has achieved 
significant accuracy compared with other models. However, current research also indicates that 
tuning BERT with new datasets can lead to further improvements in its performances for specific 
tasks. In this proposal, we suggest fine-tuning a pre-trained BERT large model (uncased) with the 
ms_marco dataset using whole word masking and incorporating a fine-tuning approach to improve its 
performances. 

The BERT model is based on the concept of pre-training, as well as fine-tuning, a NLP task. The pretraining process is based on unsupervised objectives, such as the masked language model and the 
next sentence prediction task, to learn contextual representations of words. The fine-tuning process, 
on the other hand, uses supervised training, where the learned pre-trained knowledge is combined 
with task-specific datasets to achieve beƩer performances.

MS_MARCO is a dataset consisting of a large collection of real-world queries and corresponding 
passages that aims to facilitate research in question-answer matching and ranking. It contains more 
than 1 million queries and over 8 million passages, making it one of the largest publicly available 
datasets of its kind. Each query in the dataset was generated by real users of the Bing search engine 
and is paired with relevant passages retrieved from web pages. The dataset is designed to enable 
researchers to develop and evaluate machine learning models for natural language processing tasks 
such as question answering, information retrieval, and passage ranking. The dataset also includes 
information about the relevance of passages to queries, allowing for the evaluation of ranking 
algorithms. Overall, the MS MARCO v1.2 dataset is a valuable resource for anyone interested in 
developing and evaluating algorithms for natural language understanding tasks.


---

## Project Proposal

Our proposal is to use the BERT large model (uncased) with the MS_MARCO dataset using whole 
word masking as an endpoint to answer question. For this we will go through the following steps: 
* Preparing the data: We will use the MS_MARCO dataset to fine-tune the BERT large model. We will construct a train and test set from the available dataset for fine-tuning. 
* Fine-tuning the model: We will fine-tune the pre-trained BERT model with the MS_MARCO dataset using whole word masking. We will use the Adam optimizer for training and evaluate model performance based on F1 score. 
* Evaluating the model: We will evaluate the fine-tuned model's performance based on MS_Marco's benchmarking metrics. The key evaluation measures will be the F1-score and the exact match (EM) score. 
* Deploy the model. 
* Scale the model so that we can make this application available to a lot more users. 

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with a the appropriate full IAM role arn string(s).

---

# Prerequisites

## Permissions

*If you are going to use Sagemaker in a local environment. You need access to an IAM Role with the required permissions for Sagemaker. You can find here more about it.*

In [11]:
# cell 01
import sagemaker
import boto3
import re

sess = sagemaker.Session()
prefix = 'sagemaker/DEMO-BertMarco-dm'
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

bucket=sagemaker.Session().default_bucket()
sess = sagemaker.Session(default_bucket=bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

print(bucket)
print(role)

sagemaker-us-east-1-369074678854
arn:aws:iam::369074678854:role/sagemaker-immersion-day-SageMakerExecutionRole-2UAKB1ISM2UJ


## Libraries

Now let's bring in the Python libraries that we'll use throughout the analysis

In [12]:
# cell 02
import numpy as np                                # For matrix operations and numerical processing
import pandas as pd                               # For munging tabular data
import matplotlib.pyplot as plt                   # For charts and visualizations
from IPython.display import Image                 # For displaying images in the notebook
from IPython.display import display               # For displaying outputs in the notebook
from time import gmtime, strftime                 # For labeling SageMaker models, endpoints, etc.
import sys                                        # For writing outputs to notebook
import math                                       # For ceiling function
import json                                       # For parsing hosting outputs
import os                                         # For manipulating filepath names
import sagemaker 
import zipfile     # Amazon SageMaker's Python SDK provides many helper functions
from datasets import load_dataset

---

# Data

## Import

Let's start by importing the dataset from Huggingface

In [13]:
validation_data, train_data, test_data = load_dataset('ms_marco', 'v1.1', split =['validation','train', 'test'], 
                                                        cache_dir='/media/data_files/github/website_tutorials/data')

Found cached dataset ms_marco (/media/data_files/github/website_tutorials/data/ms_marco/v1.1/1.1.0/b6a62715fa5219aea5275dd3556601004cd63945cb63e36e022f77bb3cbbca84)


  0%|          | 0/3 [00:00<?, ?it/s]

Now lets read this into a Pandas data frame and take a look.

In [23]:
df = pd.concat([pd.DataFrame(validation_data), pd.DataFrame(train_data), pd.DataFrame(test_data)],ignore_index=True)
df.to_csv('MS_Marco.csv', index=False, header=True)
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 20)         # Keep the output on one page
df

,answers,passages,query,query_id,query_type,wellFormedAnswers
0,"[Approximately $15,000 per year.]","{'is_selected': [1, 0, 0, 0, 0, 0], 'passage_t...",walgreens store sales average,9652,numeric,[]
1,"[$21,550 per year, The average hourly wage for...","{'is_selected': [0, 1, 0, 0, 0, 0, 0, 0], 'pas...",how much do bartenders make,9653,numeric,[]
2,"[A boil, also called a furuncle, is a deep fol...","{'is_selected': [0, 0, 0, 0, 0, 0, 1, 0], 'pas...",what is a furuncle boil,9654,description,[]
3,"[Detect and assess a wide range of disorders, ...","{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0], '...",what can urinalysis detect,9655,description,[]
4,"[Shigellosis, diseases of the nervous system, ...","{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0], '...",what is vitamin a used for,9656,description,[]
...,...,...,...,...,...,...
102018,[Can last 3-4 days in the fridge as long as it...,"{'is_selected': [0, 0, 0, 1, 0, 0, 0], 'passag...",how long can you keep bacon in the fridge,9647,numeric,[]
102019,[Body mass index (BMI) the weight in kilograms...,"{'is_selected': [0, 0, 0, 0, 0, 0, 1, 0, 0], '...",what is growth bmi mean in medical terms,9648,description,[]
102020,[Yes],"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0], '...",can an llc apply for more than one dba,9649,description,[]
102021,['Bisque' is a shade of White that is 23% satu...,"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0], 'pas...",bisque color definition,9650,entity,[]


## Store

We will store this natively in S3 to then process it with SageMaker Processing.

In [24]:
from sagemaker import Session

sess = Session()
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'MS_Marco_raw/MS_Marco_raw.csv')).upload_file('MS_Marco.csv')

sagemaker-us-east-1-369074678854/sagemaker/DEMO-BertMarco-dm


---
# Feature Engineering 

Here, we'll import the dataset and transform it with SageMaker Processing, which can be used to process terabytes of data in a SageMaker-managed cluster separate from the instance running your notebook server.

For this project, we're using a SageMaker prebuilt Scikit-learn container, which includes many common functions for processing data. Moreover, when the job is complete, SageMaker Processing will automatically uploads the transformed data to S3.

In [ ]:
%%writefile preprocessing.py

def _parse_args():

    parser = argparse.ArgumentParser()

    # Data, model, and output directories
    # model_dir is always passed in from SageMaker. By default this is a S3 path under the default bucket.
    parser.add_argument('--filepath', type=str, default='/opt/ml/processing/input/')
    parser.add_argument('--filename', type=str, default='bank-additional-full.csv')
    parser.add_argument('--outputpath', type=str, default='/opt/ml/processing/output/')
    parser.add_argument('--categorical_features', type=str, default='y, job, marital, education, default, housing, loan, contact, month, day_of_week, poutcome')

    return parser.parse_known_args()

if __name__=="__main__":
    # Process arguments
    args, _ = _parse_args()
    # Load data
    df = pd.read_csv(os.path.join(args.filepath, args.filename))
    # Change the value . into _
    df = df.replace(regex=r'\.', value='_')
    df = df.replace(regex=r'\_$', value='')
    # Add two new indicators
    df["no_previous_contact"] = (df["pdays"] == 999).astype(int)
    df["not_working"] = df["job"].isin(["student", "retired", "unemployed"]).astype(int)
    df = df.drop(['duration', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed'], axis=1)
    # Encode the categorical features
    df = pd.get_dummies(df)
    # Train, test, validation split
    train_data, validation_data, test_data = np.split(df.sample(frac=1, random_state=42), [int(0.7 * len(df)), int(0.9 * len(df))])   # Randomly sort the data then split out first 70%, second 20%, and last 10%
    # Local store
    pd.concat([train_data['y_yes'], train_data.drop(['y_yes','y_no'], axis=1)], axis=1).to_csv(os.path.join(args.outputpath, 'train/train.csv'), index=False, header=False)
    pd.concat([validation_data['y_yes'], validation_data.drop(['y_yes','y_no'], axis=1)], axis=1).to_csv(os.path.join(args.outputpath, 'validation/validation.csv'), index=False, header=False)
    test_data['y_yes'].to_csv(os.path.join(args.outputpath, 'test/test_y.csv'), index=False, header=False)
    test_data.drop(['y_yes','y_no'], axis=1).to_csv(os.path.join(args.outputpath, 'test/test_x.csv'), index=False, header=False)
    print("## Processing complete. Exiting.")

---
# Development environment

In [ ]:
import sagemaker.huggingface